### Day 3

Description of the puzzle [here](https://adventofcode.com/2019/day/3)

### Preliminaries

In [1]:
import numpy as np

In [2]:
with open('day_3_input.txt', 'r') as f:
    data=[line.split(',') for line in f.readlines()]

### Part 1

#### Initial attempt

**This is a good fail !**

**AKA: a good example of having misinterpreted the brief, but still having learned something.**

This was developed under the incorrect assumption that the task was to find intersections between wires only at node points in the paths, not at intermediate points along the paths.

Test case

In [3]:
t1 = ['R8','U5','L5','D3']

In [4]:
t2 = ['U7','R6','D4','L4']

Function to create vectors of wire moves

In [5]:
def position_changes(t):
    tdx = np.zeros(len(t), dtype=int)
    tdy = np.zeros(len(t), dtype=int)
    for i, el in enumerate(t):
        if el[0] == 'R':
            tdx[i] += (int(el[1::])) 
        elif el[0] == 'L':
            tdx[i] -=  (int(el[1::]))        
        if el[0] == 'U':
            tdy[i] += (int(el[1::])) 
        elif el[0] == 'D':
            tdy[i] -= (int(el[1::]))    
    return list(zip(tdx, tdy))    

In [6]:
moves_t1 = position_changes(t1)
moves_t2 = position_changes(t2)

Function to create vectors of wire node coordinates 

From: https://stackoverflow.com/a/16425408/1034648

In [7]:
def accumulate_tuples(iterable):
    accum_a = accum_b = 0
    for a, b in iterable:
        accum_a += a
        accum_b += b
        yield accum_a, accum_b

In [8]:
coord_t1 = list(accumulate_tuples(moves_t1))
coord_t2 = list(accumulate_tuples(moves_t2))

coord_t1.insert(0, (0, 0))
coord_t2.insert(0, (0, 0))

In [9]:
print(coord_t1)
print(coord_t2)

[(0, 0), (8, 0), (8, 5), (3, 5), (3, 2)]
[(0, 0), (0, 7), (6, 7), (6, 3), (2, 3)]


Find nodes at which wires intersect 

In [10]:
[np.where(a == b) for a, b in zip (coord_t1, coord_t2)]

[(array([0], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),)]

**That's a big fail above!!**

#### Working solution

With this approach we split each input into a direction and a distance and we move that distance a step at a time in the appropriate direction, to create accumulated coordinates: the wire path.

Saving those coordinates as a set, then can to find intersections of the two sets (wire 1 and wire 2), and after that `scipy.spatial.distance.cdist` to get distances from origin [0,0] of each intersection and return the minimum distance.

In [11]:
def positions(t):
    pos_set = set()
    posx = 0
    posy = 0
    for el in t:
        direction = el[0]
        distance = int(el[1:])
        for d in range(distance):
            if direction == 'R':
                posx += 1               
            elif direction == 'L':
                posx -= 1    
            if direction == 'U':
                posy += 1 
            elif direction == 'D':
                posy -= 1 
            pos_set.add((posx, posy)) 
    return (pos_set)

##### Test

In [12]:
coordt1 = positions(t1)
coordt2 = positions(t2)

In [13]:
type(coordt1)

set

In [14]:
crossings_t = np.array(list(coordt1.intersection(coordt2)))
crossings_t

array([[3, 3],
       [6, 5]])

In [15]:
from scipy.spatial.distance import cdist
distance_t = int(min(cdist(crossings_t, [[0,0]], metric='cityblock')))
distance_t

6

##### Solution

In [16]:
wire1 = data[0]
wire2 = data[1]

coord1 = positions(wire1)
coord2 = positions(wire2)

crossings = list(coord1.intersection(coord2))
distance = int(min(cdist(np.array(crossings), [[0,0]], metric='cityblock')))
distance

1519

### To do: move the if statements for directions outside of the for loop for efficiency?

### Part 2

Function to calculate cumulative distance at each step in the wire path
Then find index of first occurrence of each intersection along wires and use it to retrieve cum distances, then find shortest

Experimenting ......

In [17]:
def cum_path(t):
    cum_path = []
    for el in t:
        distance = int(el[1:])
        for d in range(distance):
            cum_path.append(d)
    return cum_path

In [18]:
print(cum_path(t1), np.cumsum(cum_path(t1)))

[0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2] [ 0  1  3  6 10 15 21 28 28 29 31 34 38 38 39 41 44 48 48 49 51]


In [19]:
idx = np.where([x == y for x in crossings_t for y in coord_t1])
idx

(array([3, 4, 7, 8], dtype=int64), array([0, 0, 1, 1], dtype=int64))

In [20]:
ls = [x for x in crossings_t]
print(ls) 

[array([3, 3]), array([6, 5])]
